In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import pickle

from Utilities.utils import *
from Utilities.plot_functions import *

# Define Experiment Parameters

In [ ]:
# Set cardiac out (CO) as the prediction variable
prediction_variable = 'CO'
variable_unit = 'L/min'
input_set = 'radial' 

# Define the prediction model
model_selection = 'ANN'

# Enable normalization 
norm_mode = False 

# Enable noise addition and select random noise range
noise_mode = False
noise_perc_lower = 5
noise_perc_upper = 5

# Enable figure saving
save_figure_boolean = False 
experiment_type = 'Data/insilico_{}_{}_{}'.format(input_set,norm_mode, noise_mode)    # Set the type of data being used in the analysis

# Load & Prepare Data

In [ ]:
dataset = select_input_dataset(norm_mode)
print('The dataset size is:',dataset.shape)

noisy_dataset = add_random_noise(dataset, noise_perc_lower, noise_perc_upper, noise_mode)
scaled_dataset = scale_data(noisy_dataset)
# print(scaled_dataset.isnull().values.any())

# Regression analysis

In [ ]:
# get_and_save_train_test_val_indices(scaled_dataset,select_test_size = 0.2,select_val_rel_size = 0.25) # only the first time

X_train,X_test,X_val,y_train,y_test,y_val = split_train_test_val_sets(scaled_dataset)
print('The size of the train set is:', X_train.shape)
print('The size of the validation set is:', X_val.shape)
print('The size of the test set is:', X_test.shape)

### Find Optimal Number of Epochs using Early Stopping Callback

In [ ]:
batch_size = 64; verbose = 0; 
# optimal_no_epochs = find_optimal_no_epochs(batch_size,dataset,X_train,X_val,y_train,y_val,prediction_variable,verbose,norm_mode)

# Alternatively, insert values manually
optimal_no_epochs = 61

In [ ]:
model, y_pred = artificial_neural_network(batch_size,optimal_no_epochs,X_train,X_test,y_train,y_test,0) 

rescaled_y_test = rescale_values(y_test, prediction_variable, dataset)
rescaled_y_pred = rescale_values(y_pred, prediction_variable, dataset).flatten()


### Print Results

In [ ]:
print_results(rescaled_y_test,rescaled_y_pred,variable_unit)

### Plot Results

In [ ]:
plot_data(rescaled_y_test,rescaled_y_pred,experiment_type,prediction_variable,model_selection,save_figure_boolean)

# Run experiments using the saved models

In [ ]:
# Load pre-developed models from disk
if norm_mode:
    loaded_model = pickle.load(open('TrainedModels/uncal_data_trained_model.sav', 'rb'))
else:
    loaded_model = pickle.load(open('TrainedModels/cal_data_trained_model.sav', 'rb'))
        
loaded_model.fit(X_test, y_test)
y_pred = loaded_model.predict(X_test)

rescaled_y_test = rescale_values(y_test, prediction_variable, dataset)
rescaled_y_pred = rescale_values(y_pred, prediction_variable, dataset).flatten()
    
print_results(rescaled_y_test,rescaled_y_pred,variable_unit)